In [1]:
# State the parameters of the pipeline

disk = '/mnt/pre_epi_seizures/'
baseline_files = 'h5_files/processing_datasets/baseline_datasets_new'
seizure_files = 'h5_files/processing_datasets/seizure_datasets_new'


# Exploratory Data Analysis
from classification.load_for_class import *
import convertpandas as cv_pd
import pandas as pd
import sklearn.preprocessing as pp
from sklearn.kernel_approximation import RBFSampler
from interim_processing import *
import iopes

# Set parameters of analyssis
feature_slot = 'hrv_time_features'

# Chose hyper-parameters of acquisition (ix of saved files)
hyper_param = 0


patient_list = [3]
lead_list = ['ECG-']
scaler = pp.StandardScaler()
rbf_feature = RBFSampler(gamma=1, random_state=1)
interim_processing = [scaler]
hist_bins = None
dist = None
flag_hist = False
flag_andrews = False
flag_series = False
flag_box = False
flag_pair = False
assign_baseline = 'assign_equal_baseline_seizure'
label_struct = {
                        'inter_ictal':{
                            'label': 'Inter-Ictal Data Points',
                            'color': 'blue',
                            'intervals_samples': [(1000 * 0 * 60, 1000 * 0 * 60 )]
                                     },
                        'pre_ictal':{
                            'label': 'Pre-Ictal data points',
                            'color': 'yellow',
                            'intervals_samples': [(1000 * 20 * 60, 1000 * 49 * 60 )]
                                     },
                        'ictal':{
                            'label': 'Ictal data points',
                            'color': 'red',
                            'intervals_samples': [(1000 * 50 * 60, 1000 * 70 * 60 )]
                                 },
                        }
baseline_label_struct = {
                         'baseline':{
                            'label': 'Baseline Data Points',
                            'color': 'green',
                            'intervals_samples': [(0, 1000 * 120 * 60 )]
                                    },
                        }

/home/franciscosargo/.virtualenvs/pre_epi_seizures/local/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
eda_dir = 'EDAnalysis/'

eda_id = iopes.get_eda_params_path(disk=disk,
                              eda_dir=eda_dir,
                        patient_list = patient_list,
                          lead_list = lead_list,
                            scaler = scaler,
                            interim_processing = interim_processing,
                            hist_bins = hist_bins,
                            dist = dist,
                            flag_hist = flag_hist,
                            flag_andrews = flag_andrews,
                            flag_series = flag_series,
                            flag_box = flag_box,
                            assign_baseline = assign_baseline,
                            label_struct = label_struct,
                            baseline_label_struct = baseline_label_struct)
path = disk + eda_dir + eda_id + '/'

('flag_box', False)('flag_series', False)('dist', None)('scaler', StandardScaler(copy=True, with_mean=True, with_std=True))('lead_list', ['ECG-'])('interim_processing', [StandardScaler(copy=True, with_mean=True, with_std=True)])('flag_hist', False)('assign_baseline', 'assign_equal_baseline_seizure')('label_struct', {'pre_ictal': {'color': 'yellow', 'intervals_samples': [(1200000, 2940000)], 'label': 'Pre-Ictal data points'}, 'inter_ictal': {'color': 'blue', 'intervals_samples': [(0, 0)], 'label': 'Inter-Ictal Data Points'}, 'ictal': {'color': 'red', 'intervals_samples': [(3000000, 4200000)], 'label': 'Ictal data points'}})('baseline_label_struct', {'baseline': {'color': 'green', 'intervals_samples': [(0, 7200000)], 'label': 'Baseline Data Points'}})('flag_andrews', False)('patient_list', [3])('hist_bins', None)


In [3]:
if not os.path.exists(path):
    os.mkdir(path)

In [4]:
# Ingest Seizure Data
path_to_load = disk + seizure_files + '.h5'
path_to_map = disk + seizure_files + '_map.txt'

# Feature group to analyse -- point of entry
feature_name = get_feature_group_name_list(path_to_map,
                                               feature_slot)[hyper_param]

print feature_name


seizure_data = cv_pd.convert_to_pandas(path_to_load, path_to_map,
                        patient_list, feature_name,
                        lead_list, label_struct)
seizure_data

IndexError: list index out of range

In [ ]:
# Ingest Baseline Data

# set Labeling structure
path_to_load = disk + baseline_files + '.h5'
path_to_map = disk + baseline_files + '_map.txt'



# Feature group to analyse -- point of entry
feature_name = get_feature_group_name_list(path_to_map,
                                               feature_slot)[hyper_param]

baseline_data = cv_pd.convert_to_pandas(path_to_load, path_to_map,
                        patient_list, feature_name,
                        lead_list, baseline_label_struct)

baseline_data

In [ ]:

baseline_data = baseline_data.dropna(axis=0, how='any').reset_index(drop=True)

baseline_data = globals()[assign_baseline](baseline_data,
                                          seizure_data,
                                         'seizure_nr',
                                         'patient_nr')
#print 'Done'

seizure_data = seizure_data.dropna(axis=0, how='any').reset_index(drop=True)

data = pd.concat([seizure_data, baseline_data],
                 ignore_index=True)

#except Exception as e:
#    print e
#    data = seizure_data
    
    
    

In [ ]:
cv_pd.add_seizure_types(data,
                        'patient_nr',
                        'seizure_nr',
                        'types_of_seizure',
                        'location')

In [ ]:
# state the Data metafeatures
metafeatures = ['patient_nr', 'seizure_nr', 'time_sample', 'label', 'color', 'types_of_seizure', 'location']
features = [column
            for column in data.columns
            if column not in metafeatures]

# Drop missing values
data = data.dropna(axis=0, how='any').reset_index(drop=True)

In [ ]:
# Interim process the data
for step in interim_processing:
    X = data[features]

    X_norm_np = step.fit_transform(X)

    #X_norm = pd.DataFrame(X_norm_np, columns=X.columns)

    data[features] = X_norm_np

In [ ]:
# MLP for Pima Indians Dataset with 10-fold cross validation via sklearn
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
import numpy
 
# Function to create model, required for KerasClassifier
def dense_network(output_dim,
                  input_dim,
                  hidden_layers_nr,
                  hidden_nodes_per_layer,
                  hidden_nodes_activation):
    # create model
    model = Sequential()
    
    # Create Hidden Layer Topology
    for layer in xrange(0, hidden_layers_nr):
        model.add(Dense(hidden_nodes_per_layer,
                        input_dim=input_dim,
                        activation=hidden_nodes_activation))
 
    # Add output layer
    model.add(Dense(output_dim, activation='softmax'))
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


In [ ]:
# Modelation
import sklearn.svm as svm
from sklearn.model_selection import *
from sklearn.pipeline import *
import sklearn.naive_bayes as nb
from sklearn.feature_selection import *
from sklearn.neural_network import *
from keras.utils import np_utils

# Group the data
data_groups = data.groupby(['patient_nr',
                              'seizure_nr',
                              'types_of_seizure',
                              'location'])

group_id = 'seizure_nr'

data_groups_list = list(data_groups)

# prepare data for classification - watch out for memory concerns
X = data[features]
y = data['label']
groups = data[group_id]


# choose Pipeline **Details in pipelines.py 
#pipe = Pipeline([('estimator', svm.SVC())])
#pipe = Pipeline( [('GaussNB', nb.GaussianNB())])


pipe = Pipeline( [('ANN', KerasClassifier(build_fn=dense_network,
                                                input_dim = len(features),
                                                output_dim = len(y.unique()),
                                                verbose=0))])
pipe_id = []
#pipe = Pipeline([('estimator', MLPClassifier())])

# choose parameter search method *coherent with Pipeline steps
param_grid = [
               {'estimator__C': [2**i for i in xrange(-5, 11)],
                'estimator__gamma':[2**i for i in xrange(-15, 1)]},
                  ]

param_grid = [{'__activation': ['relu'],
               'estimator__solver':['adam']}]


#param_grid = [
#               {'GaussNB__priors': [None]},
#                  ]

param_grid = [
              {'ANN__epochs': [2000],
               'ANN__batch_size': [10**4],
               'ANN__hidden_layers_nr': [i for i in xrange(1, 5)],
               'ANN__hidden_nodes_per_layer': [2**i for i in xrange(1, 5)],
               'ANN__hidden_nodes_activation': ['relu']
              }
                ]

# define cross-validation strategy 
cv_out = LeavePGroupsOut(n_groups=1)
cv_in = LeavePGroupsOut(n_groups=1)

# choose scoring
scoring = ['f1_micro']

# choose wether to perform new computation
compute_all_new = True

search_function = GridSearchCV

hyper_param_heat = False
# Get path to save the results
#full_path = get_full_pipeline_name(path_to_save,
#                                   file_to_save,
#                                   pipe, 
#                                   scoring,
#                                   param_grid,
#                                   feature_names,
#                                   cv_out,
#                                   cv_in,
#                                   trial)



# plot_scatter(path_to_save, data_struct, class_metadata)

# plot_full(file_to_save, data_struct, class_metadata)

# Create directory to save results
#make_dir(full_path)


In [ ]:
clf_id = iopes.get_eda_params_path(disk=disk,
                                   eda_dir=eda_dir + '/' + eda_id + '/' ,
                                   pipe = str(pipe),
                                   param_grid = param_grid,
                                   cv_out = cv_out,
                                   cv_in = cv_in,
                                   scoring = scoring,
                                   search_function = search_function,
                                   group_id=group_id)

path_to_save = disk + eda_dir + eda_id + '/' + clf_id

In [ ]:
path_to_save = path


import classification.eda.hist as plt_hist
import classification.eda.andrews as plt_and
import classification.eda.series as plt_ts
import classification.eda.box as plt_box
import classification.eda.scatter as plt_sc
import matplotlib.pyplot as plt




for data_patient_seizure in data_groups_list:

    if flag_hist:
        plt_hist.histogram(path_to_save,
                                data_patient_seizure[1],
                                data_patient_seizure[0],
                                features,
                                'time_sample',
                                'patient_nr',
                                'seizure_nr',
                                'label',
                                'color',
                                 bins=hist_bins,
                                 dist=dist)

    
    if flag_series:
        plt_ts.time_series_plot(path_to_save, data_patient_seizure[1],
                                features,
                                'time_sample',
                                'patient_nr',
                                'seizure_nr',
                                'label',
                                'color')
    if flag_andrews:
        plt_and.andrews_curves(path_to_save,
                                data_patient_seizure[1],
                                data_patient_seizure[0],
                                features,
                                'time_sample',
                                'patient_nr',
                                'seizure_nr',
                                'label',
                                'color')
    if flag_box:
        plt_box.box_plot(path_to_save,
                        data_patient_seizure[1],
                        data_patient_seizure[0],
                        features,
                        'time_sample',
                        'patient_nr',
                        'seizure_nr',
                        'label',
                        'color')
        
    if flag_pair:

        plt_sc.pair_plot(path_to_save,
                        data_patient_seizure[1],
                        data_patient_seizure[0],
                        features,
                        'time_sample',
                        'patient_nr',
                        'seizure_nr',
                        'label',
                        'color')
    

In [ ]:
import classification.cross_validation as cv

# ***********************************Learning****************************
# Learn from data_struct using nested cross_validation
# learninig is an optimization and respective test results
# for each partition of the dataset according to cv_out
learning_results = cv.nested_cross_validation(path_to_save,
                                       X,y, groups,
                                       pipe,
                                       param_grid, scoring,
                                       compute_all_new, cv_out, cv_in,
                                       search_function)
#************************************************************************
groups = data_groups.groups.keys()

for learning_result, group in zip(learning_results, groups):
        learning_result['group'] = group
        
cv_object = learning_results
cv_object



In [ ]:
import seaborn as sns
import itertools

if hyper_param_heat:
    for clf, test in learning_results:
        print clf.best_estimator_
        cv_results = clf.cv_results_
        keys = cv_results.keys()

        for grid in param_grid:
            estimator_params = [key 
                                for key in grid.keys()
                                if 'estimator' in key]
            print estimator_params
            param_bi_comb = itertools.combinations(estimator_params, r=2)

            for bi_comb in param_bi_comb:

                for key in keys:

                    if 'param' not in key and 'rank' not in key:
                        metric = cv_results[key]
                        df = metric.reshape(len([2**i for i in xrange(-5, 11)]), len([2**i for i in xrange(-15, 1)]))

                        #df = pd.DataFrame(df, columns=[str(2**i) for i in xrange(-15, 1)])
                        #df['ix'] = [str(2**i) for i in xrange(-5, 11)]
                        #df.set_index('ix')

                        plt.figure()
                        sns.heatmap(df,
                                    xticklabels=[2**i for i in xrange(-5, 11)],
                                    yticklabels=[2**i for i in xrange(-15, 1)],
                                    cbar_kws={'label': key},
                                   )
                        plt.title(key)
                        plt.xlabel(bi_comb[0])
                        plt.ylabel(bi_comb[1])
                        #plt.savefig(path_to_save + '/' + key)
                        plt.show()


In [ ]:
from io import StringIO
import pandas as pd
import numpy as np
import sklearn
def parse_classification_report(classification_report):
    """Parses sklearn classification report into a pandas dataframe."""
    return pd.read_fwf(StringIO(classification_report),lineterminator='\n', index_col=0, colspecs=[(0,22),(22,32),(32,42),(42,52), (52, 62)]).dropna()

In [ ]:
from sklearn.metrics import *


def generate_classification_report(learning_results):
    # report each test from each fold of the cross_validation scheme
    report = pd.concat(map(_generate_classification_report, learning_results))
     
    # format
    report = report.reset_index(level=0, drop=True)
    return report

def _generate_classification_report(learning_result):

    # Get the parameters
    best_estimator = learning_result['best_estimator']
    best_params = learning_result['best_params']
    
    # Get the Test results
    y_test = learning_result['y_test']
    y_pred = learning_result['y_pred']
    
    # Get the test group
    group = learning_result['group']

    # Convert Test result to pandas daraframe
    classification_report_df = parse_classification_report(classification_report(y_test, y_pred, digits=4))
    classification_report_df['Labels'] = classification_report_df.index
    classification_report_df = classification_report_df.reset_index(drop=True)
    classification_report_df['Test Group'] = str(group)
    classification_report_df['Model'] = str(best_estimator.named_steps.keys())
    classification_report_df['Best Parameters'] = str(best_params)

    # Set Collumns as indexes of the dataframe
    classification_report_df.set_index(['Model',
                                        'Best Parameters',
                                        'Test Group',
                                        'Labels'] , append=True, inplace=True)

    return classification_report_df




In [ ]:
report = generate_classification_report(cv_object)
report

In [ ]:
string = report.to_latex()
print string.replace('\_', " ")

In [ ]:
import numpy as np
import pandas as pd
arrays = [['bar', 'bar', 'baz', 'baz', 'foo', 'foo', 'qux', 'qux'],['one', 'two', 'one', 'two', 'one', 'two', 'one', 'two']]
tuples = list(zip(*arrays))
index = pd.MultiIndex.from_tuples(tuples, names=['first', 'second'])
s = pd.DataFrame(np.random.randn(8, 4), index=arrays)
s


In [ ]:
print s.to_latex()

In [ ]:
del classification_report

In [ ]:
best_params

In [ ]:
learning_results